# Ejemplo de `word2vec` con `gensim`


En la siguiente celda, importamos las librerías necesarias y configuramos los mensajes de los logs.

In [1]:
import gensim, logging, os
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Entrenamiento de un modelo

Implemento una clase `Corpus` con un iterador sobre un directorio que contiene ficheros de texto. Utilizaré una instancia de `Corpus` para poder procesar de manera más eficiente una colección, sin necesidad de cargarlo previamente en memoria.

In [ ]:
class Corpus(object):
    '''Clase Corpus que permite leer de manera secuencial un directorio de documentos de texto'''
    
    def __init__(self, directorio):
        self.directory = directorio

    def __iter__(self):
        for fichero in os.listdir(self.directory):
            for linea in open(os.path.join(self.directory, fichero)):
                yield linea.split()


`CORPUSDIR` contiene una colección de noticias en español (normalizada previamente a minúsculas y sin signos de puntuación) con alrededor de 150 millones de palabras. Entrenamos un modelo en un solo paso, ignorando aquellos tokens que aparecen menos de 10 veces (ignorando erratas, básicamente), para construir vectores de 200 dimensiones. 

In [ ]:
CORPUSDIR = 'PATH_TO_YOUR_CORPUS_DIRECTORY'
oraciones = Corpus(CORPUSDIR)
model = gensim.models.Word2Vec(oraciones, min_count=10, size=200, workers=2)

Una vez completado el entrenamiento (después de casi 30 minutos), guardamos el modelo en disco. 

In [ ]:
model.save('PATH_TO_YOUR_MODEL.w2v')

En el futuro, podremos utilizar este modelo cargándolo en memoria con la instrucción:

In [2]:
model = gensim.models.Word2Vec.load('/data/w2v/eswiki-280.w2v')

2019-03-23 12:54:23,608 : INFO : loading Word2Vec object from /data/w2v/eswiki-280.w2v
2019-03-23 12:54:26,955 : INFO : loading wv recursively from /data/w2v/eswiki-280.w2v.wv.* with mmap=None
2019-03-23 12:54:26,959 : INFO : loading syn0 from /data/w2v/eswiki-280.w2v.wv.syn0.npy with mmap=None
2019-03-23 12:54:27,698 : INFO : setting ignored attribute syn0norm to None
2019-03-23 12:54:27,702 : INFO : loading syn1neg from /data/w2v/eswiki-280.w2v.syn1neg.npy with mmap=None
2019-03-23 12:54:28,958 : INFO : Model saved using code from earlier Gensim Version. Re-loading old model in a compatible way.
2019-03-23 12:54:28,962 : INFO : loading Word2Vec object from /data/w2v/eswiki-280.w2v
2019-03-23 12:54:32,672 : INFO : loading wv recursively from /data/w2v/eswiki-280.w2v.wv.* with mmap=None
2019-03-23 12:54:32,676 : INFO : loading syn0 from /data/w2v/eswiki-280.w2v.wv.syn0.npy with mmap=None
2019-03-23 12:54:33,915 : INFO : setting ignored attribute syn0norm to None
2019-03-23 12:54:33,919

## Probando nuestro modelo

El objeto `model` contiene una enorme matriz de números: una tabla, donde cada fila es uno de los términos del vocabulario reconocido y cada columna es una de las características que permiten modelar el significado de dicho término.

En nuestro modelo, tal y como está entrenado, tenemos más de 26 millones de términos:

In [3]:
print(model.corpus_count)

24579643


Cada término del vocabulario está representado como un vector con 150 dimensiones: 105 características. Podemos acceder al vector de un término concreto:

In [4]:
print(model['azul'], '\n')

print(model['verde'], '\n')

print(model['microsoft'])

[ 4.96991217e-01  8.55721772e-01  6.82879806e-01 -4.14421827e-01
  1.72151077e+00 -8.15706775e-02 -3.84779096e-01  2.39795947e+00
 -1.08219063e+00  2.05436572e-01 -1.08851862e+00 -1.52230406e+00
 -3.02234702e-02  1.71446276e+00  9.03053999e-01  5.00934906e-02
 -2.77492136e-01  6.52989566e-01  2.99589902e-01 -1.67158830e+00
  7.22501874e-01  1.72892824e-01  2.41062713e+00 -1.96764266e+00
 -2.54679441e+00  3.38947445e-01  3.77230197e-01 -2.00717807e+00
 -1.37975141e-01  1.50688481e+00  9.42857385e-01  1.03348625e+00
 -3.24400783e+00  3.71784419e-01 -9.88212645e-01 -1.13850737e+00
  1.25355625e+00 -1.97994685e+00 -1.87854564e+00  1.29538226e+00
 -7.06246436e-01  1.55038989e+00  3.01234293e+00  7.06765711e-01
 -4.44420457e-01 -1.43686986e+00 -2.71735215e+00  1.69624233e+00
  1.35413718e+00  6.97062194e-01  1.51717079e+00  1.42068231e+00
  4.78212088e-01  2.25567269e+00  6.34011805e-01  1.51526082e+00
 -5.22151649e-01 -1.78797102e+00  2.67135262e-01  4.13583934e-01
 -4.50551808e-01  6.93174

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


Estos vectores no nos dicen mucho, salvo que contienen números muy pequeños :-/

El mismo objeto `model` permite acceder a una serie de funcionalidades ya implementadas que nos van a permitir evaluar formal e informalmente el modelo. Por el momento, nos contentamos con los segundo: vamos a revisar visualmente los significados que nuestro modelo ha aprendido por su cuenta. 

Podemos calcular la similitud semántica entre dos términos usando el método `similarity`, que nos devuelve un número entre 0 y 1:

In [6]:
print('hombre - mujer', model.wv.similarity('hombre', 'mujer'))

print('perro - gato', model.wv.similarity('perro', 'gato'))

print('gato - periódico', model.wv.similarity('gato', 'periódico'))

print('febrero - azul', model.wv.similarity('febrero', 'azul'))

hombre - mujer 0.45342360344820054
perro - gato 0.7835531458527887
gato - periódico 0.16155739656526133
febrero - azul -0.006579301654385549


Podemos seleccionar el término que no encaja a partir de una determinada lista de términos usando el método `doesnt_match`:

In [7]:
lista1 = 'madrid barcelona gonzález washington'.split()
print('en la lista', ' '.join(lista1), 'sobra:', model.wv.doesnt_match(lista1))

lista2 = 'psoe pp ciu ronaldo'.split()
print('en la lista', ' '.join(lista2), 'sobra:', model.wv.doesnt_match(lista2))

lista3 = 'publicaron declararon soy negaron'.split()
print('en la lista', ' '.join(lista3), 'sobra:', model.wv.doesnt_match(lista3))

lista4 = 'homero saturno cervantes shakespeare cela'.split()
print('en la lista', ' '.join(lista4), 'sobra:', model.wv.doesnt_match(lista4))

lista5 = 'madrid barcelona alpedrete marsella'.split()
print('en la lista', ' '.join(lista5), 'sobra:', model.wv.doesnt_match(lista5))

2019-03-23 13:00:40,577 : INFO : precomputing L2-norms of word weight vectors


en la lista madrid barcelona gonzález washington sobra: washington
en la lista psoe pp ciu ronaldo sobra: ronaldo
en la lista publicaron declararon soy negaron sobra: soy
en la lista homero saturno cervantes shakespeare cela sobra: saturno
en la lista madrid barcelona alpedrete marsella sobra: alpedrete


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:618: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


Podemos buscar los términos más similares usando el método `most_similar` de nuestro modelo:

In [8]:
terminos = 'psoe chicago rajoy enero amarillo microsoft iberia ronaldo messi atlético 1992'.split()

for t in terminos:
    print(t, '==>', model.wv.most_similar(t), '\n')

psoe ==> [('pnv', 0.8303512930870056), ('psc', 0.8154621124267578), ('pce', 0.8059691190719604), ('pse-ee', 0.7822012305259705), ('bng', 0.7804697155952454), ('pspv-psoe', 0.7800180912017822), ('psc-psoe', 0.7689586877822876), ('pp', 0.7646921873092651), ('prc', 0.761844277381897), ('iu', 0.7561599612236023)] 

chicago ==> [('boston', 0.7530511617660522), ('york', 0.7354574799537659), ('filadelfia', 0.699373722076416), ('pittsburgh', 0.6872875094413757), ('cincinnati', 0.684821367263794), ('detroit', 0.6755380630493164), ('baltimore', 0.6473920345306396), ('brooklyn', 0.6452175378799438), ('dallas', 0.6409173607826233), ('minneapolis', 0.6332350969314575)] 

rajoy ==> [('ruiz-funes', 0.5886357426643372), ('fragueiro', 0.5773149728775024), ('barbacid', 0.5633131265640259), ('azaña', 0.5572462677955627), ('ruiz-esquide', 0.5542961359024048), ('cañardo', 0.5493910312652588), ('lagasca', 0.5486473441123962), ('iúdica', 0.5368566513061523), ('comense', 0.5313370227813721), ('alfonsín', 0.52

Con el mismo método `most_similar` podemos combinar vectores de palabras tratando de jugar con los rasgos semánticos de cada una de ellas para descubrir nuevas relaciones.

In [9]:
print('mujer que ejerce la autoridad en una alcaldía ==> alcalde + mujer - hombre')
most_similar = model.wv.most_similar(positive=['alcalde', 'mujer'], negative=['hombre'], topn=3)
for item in most_similar:
    print(item)

print('monarca soberano ==> reina + hombre - mujer')    
most_similar = model.wv.most_similar(positive=['reina', 'hombre'], negative=['mujer'], topn=3)
for item in most_similar:
    print(item)
    
print('capital de Alemania ==> moscú + alemania - rusia')
most_similar = model.wv.most_similar(positive=['moscú', 'alemania'], negative=['rusia'], topn=3)
for item in most_similar:
    print(item)

print('presidente de Francia ==> rajoy + francia - españa')
most_similar = model.wv.most_similar(positive=['mariano', 'francia'], negative=['españa'], topn=3)
for item in most_similar:
    print(item)

mujer que ejerce la autoridad en una alcaldía ==> alcalde + mujer - hombre
('alcaldesa', 0.729141891002655)
('concejala', 0.6247143745422363)
('regidora', 0.6115264892578125)
monarca soberano ==> reina + hombre - mujer
('rey', 0.5994666814804077)
('monarca', 0.5496434569358826)
('príncipe', 0.4957609176635742)
capital de Alemania ==> moscú + alemania - rusia
('berlín', 0.7999091744422913)
('múnich', 0.7739257216453552)
('hamburgo', 0.7248113751411438)
presidente de Francia ==> rajoy + francia - españa
('manuel', 0.5396797060966492)
('enrique', 0.5293997526168823)
('remigio', 0.5291164517402649)


In [ ]:
most_similar = model.wv.most_similar(positive=['obama', 'francia'], negative=['eeuu'], topn=3)
for item in most_similar:
    print(item)